# FastCDC - Python Optimization Study

## Imports

In [15]:
import ctypes
import os
from codetiming import Timer
import humanize
import cython
import random
from array import array
from concurrent.futures import ThreadPoolExecutor
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


## System Block Size

In [16]:
def get_block_size_win():
    """Check storage cluster/sector sizes"""
    sectorsPerCluster = ctypes.c_ulonglong(0)
    bytesPerSector = ctypes.c_ulonglong(0)
    rootPathName = ctypes.c_wchar_p(u"C:\\")

    ctypes.windll.kernel32.GetDiskFreeSpaceW(rootPathName,
        ctypes.pointer(sectorsPerCluster),
        ctypes.pointer(bytesPerSector),
        None,
        None,
    )
    return sectorsPerCluster.value, bytesPerSector.value
get_block_size_win()

(8, 512)

## Create Dummy File

In [17]:
DUMMY="dummy.bin"

def create_dummy_file(size=int(1e+9)):
    """Create a Random Data File"""
    print(f'Generate {humanize.naturalsize(size)} {DUMMY}')
    with open(DUMMY, "wb") as outf:
        outf.write(os.urandom(size))

create_dummy_file()

Generate 1.0 GB dummy.bin


# File Read

## Benchmark Read Function

In [18]:
def benchmark_read(func, chunk_size):
    func_name = func.__name__
    num_bytes = os.path.getsize(DUMMY)
    file_size = humanize.naturalsize(num_bytes)
    t = Timer(logger=None)
    t.start()
    result = func(chunk_size)
    t.stop()
    print(f"{file_size} with {humanize.naturalsize(chunk_size)} chunk size with {func_name} : {humanize.naturalsize(num_bytes / t.last)}/s (result = {result})")

## Fixed Size Chunked Reader

In [19]:
def read_chunked(chunk_size=1024*32):
    with open(DUMMY, 'rb') as f:
        data = f.read(chunk_size)
        x = None
        while data:
            x = data
            data = f.read(chunk_size)
    return x[-1]

In [20]:
benchmark_read(read_chunked, chunk_size=1024)
benchmark_read(read_chunked, chunk_size=1024*8)
benchmark_read(read_chunked, chunk_size=1024*16)
benchmark_read(read_chunked, chunk_size=1024*32)
benchmark_read(read_chunked, chunk_size=1024*64)
benchmark_read(read_chunked, chunk_size=1024*128)
benchmark_read(read_chunked, chunk_size=1024*256)
benchmark_read(read_chunked, chunk_size=1024*512)
benchmark_read(read_chunked, chunk_size=1024*1024)

1.0 GB with 1.0 kB chunk size with read_chunked : 1.4 GB/s (result = 36)
1.0 GB with 8.2 kB chunk size with read_chunked : 1.8 GB/s (result = 36)
1.0 GB with 16.4 kB chunk size with read_chunked : 2.7 GB/s (result = 36)
1.0 GB with 32.8 kB chunk size with read_chunked : 4.0 GB/s (result = 36)
1.0 GB with 65.5 kB chunk size with read_chunked : 5.2 GB/s (result = 36)
1.0 GB with 131.1 kB chunk size with read_chunked : 5.9 GB/s (result = 36)
1.0 GB with 262.1 kB chunk size with read_chunked : 6.4 GB/s (result = 36)
1.0 GB with 524.3 kB chunk size with read_chunked : 6.7 GB/s (result = 36)
1.0 GB with 1.0 MB chunk size with read_chunked : 2.4 GB/s (result = 36)


# Chunking Operations 

## Simple Chunking Operations

In [21]:
GEAR = [
  1553318008, 574654857,  759734804,  310648967,  1393527547, 1195718329,
  694400241,  1154184075, 1319583805, 1298164590, 122602963,  989043992,
  1918895050, 933636724,  1369634190, 1963341198, 1565176104, 1296753019,
  1105746212, 1191982839, 1195494369, 29065008,   1635524067, 722221599,
  1355059059, 564669751,  1620421856, 1100048288, 1018120624, 1087284781,
  1723604070, 1415454125, 737834957,  1854265892, 1605418437, 1697446953,
  973791659,  674750707,  1669838606, 320299026,  1130545851, 1725494449,
  939321396,  748475270,  554975894,  1651665064, 1695413559, 671470969,
  992078781,  1935142196, 1062778243, 1901125066, 1935811166, 1644847216,
  744420649,  2068980838, 1988851904, 1263854878, 1979320293, 111370182,
  817303588,  478553825,  694867320,  685227566,  345022554,  2095989693,
  1770739427, 165413158,  1322704750, 46251975,   710520147,  700507188,
  2104251000, 1350123687, 1593227923, 1756802846, 1179873910, 1629210470,
  358373501,  807118919,  751426983,  172199468,  174707988,  1951167187,
  1328704411, 2129871494, 1242495143, 1793093310, 1721521010, 306195915,
  1609230749, 1992815783, 1790818204, 234528824,  551692332,  1930351755,
  110996527,  378457918,  638641695,  743517326,  368806918,  1583529078,
  1767199029, 182158924,  1114175764, 882553770,  552467890,  1366456705,
  934589400,  1574008098, 1798094820, 1548210079, 821697741,  601807702,
  332526858,  1693310695, 136360183,  1189114632, 506273277,  397438002,
  620771032,  676183860,  1747529440, 909035644,  142389739,  1991534368,
  272707803,  1905681287, 1210958911, 596176677,  1380009185, 1153270606,
  1150188963, 1067903737, 1020928348, 978324723,  962376754,  1368724127,
  1133797255, 1367747748, 1458212849, 537933020,  1295159285, 2104731913,
  1647629177, 1691336604, 922114202,  170715530,  1608833393, 62657989,
  1140989235, 381784875,  928003604,  449509021,  1057208185, 1239816707,
  525522922,  476962140,  102897870,  132620570,  419788154,  2095057491,
  1240747817, 1271689397, 973007445,  1380110056, 1021668229, 12064370,
  1186917580, 1017163094, 597085928,  2018803520, 1795688603, 1722115921,
  2015264326, 506263638,  1002517905, 1229603330, 1376031959, 763839898,
  1970623926, 1109937345, 524780807,  1976131071, 905940439,  1313298413,
  772929676,  1578848328, 1108240025, 577439381,  1293318580, 1512203375,
  371003697,  308046041,  320070446,  1252546340, 568098497,  1341794814,
  1922466690, 480833267,  1060838440, 969079660,  1836468543, 2049091118,
  2023431210, 383830867,  2112679659, 231203270,  1551220541, 1377927987,
  275637462,  2110145570, 1700335604, 738389040,  1688841319, 1506456297,
  1243730675, 258043479,  599084776,  41093802,   792486733,  1897397356,
  28077829,   1520357900, 361516586,  1119263216, 209458355,  45979201,
  363681532,  477245280,  2107748241, 601938891,  244572459,  1689418013,
  1141711990, 1485744349, 1181066840, 1950794776, 410494836,  1445347454,
  2137242950, 852679640,  1014566730, 1999335993, 1871390758, 1736439305,
  231222289,  603972436,  783045542,  370384393,  184356284,  709706295,
  1453549767, 591603172,  768512391,  854125182
]

In [22]:
def chunker_simple(data):
    pattern = 0
    mask = 32767
    c = 0
    for b in data:
        pattern = (pattern >> 1) + GEAR[b]
        if not pattern & mask:
            c+=1
    return c

## Cython Chunking Operations

In [23]:
%%cython
cimport cython
from libc.stdint cimport uint32_t, uint8_t

cdef uint32_t[256] GEAR_C = [
  1553318008, 574654857,  759734804,  310648967,  1393527547, 1195718329,
  694400241,  1154184075, 1319583805, 1298164590, 122602963,  989043992,
  1918895050, 933636724,  1369634190, 1963341198, 1565176104, 1296753019,
  1105746212, 1191982839, 1195494369, 29065008,   1635524067, 722221599,
  1355059059, 564669751,  1620421856, 1100048288, 1018120624, 1087284781,
  1723604070, 1415454125, 737834957,  1854265892, 1605418437, 1697446953,
  973791659,  674750707,  1669838606, 320299026,  1130545851, 1725494449,
  939321396,  748475270,  554975894,  1651665064, 1695413559, 671470969,
  992078781,  1935142196, 1062778243, 1901125066, 1935811166, 1644847216,
  744420649,  2068980838, 1988851904, 1263854878, 1979320293, 111370182,
  817303588,  478553825,  694867320,  685227566,  345022554,  2095989693,
  1770739427, 165413158,  1322704750, 46251975,   710520147,  700507188,
  2104251000, 1350123687, 1593227923, 1756802846, 1179873910, 1629210470,
  358373501,  807118919,  751426983,  172199468,  174707988,  1951167187,
  1328704411, 2129871494, 1242495143, 1793093310, 1721521010, 306195915,
  1609230749, 1992815783, 1790818204, 234528824,  551692332,  1930351755,
  110996527,  378457918,  638641695,  743517326,  368806918,  1583529078,
  1767199029, 182158924,  1114175764, 882553770,  552467890,  1366456705,
  934589400,  1574008098, 1798094820, 1548210079, 821697741,  601807702,
  332526858,  1693310695, 136360183,  1189114632, 506273277,  397438002,
  620771032,  676183860,  1747529440, 909035644,  142389739,  1991534368,
  272707803,  1905681287, 1210958911, 596176677,  1380009185, 1153270606,
  1150188963, 1067903737, 1020928348, 978324723,  962376754,  1368724127,
  1133797255, 1367747748, 1458212849, 537933020,  1295159285, 2104731913,
  1647629177, 1691336604, 922114202,  170715530,  1608833393, 62657989,
  1140989235, 381784875,  928003604,  449509021,  1057208185, 1239816707,
  525522922,  476962140,  102897870,  132620570,  419788154,  2095057491,
  1240747817, 1271689397, 973007445,  1380110056, 1021668229, 12064370,
  1186917580, 1017163094, 597085928,  2018803520, 1795688603, 1722115921,
  2015264326, 506263638,  1002517905, 1229603330, 1376031959, 763839898,
  1970623926, 1109937345, 524780807,  1976131071, 905940439,  1313298413,
  772929676,  1578848328, 1108240025, 577439381,  1293318580, 1512203375,
  371003697,  308046041,  320070446,  1252546340, 568098497,  1341794814,
  1922466690, 480833267,  1060838440, 969079660,  1836468543, 2049091118,
  2023431210, 383830867,  2112679659, 231203270,  1551220541, 1377927987,
  275637462,  2110145570, 1700335604, 738389040,  1688841319, 1506456297,
  1243730675, 258043479,  599084776,  41093802,   792486733,  1897397356,
  28077829,   1520357900, 361516586,  1119263216, 209458355,  45979201,
  363681532,  477245280,  2107748241, 601938891,  244572459,  1689418013,
  1141711990, 1485744349, 1181066840, 1950794776, 410494836,  1445347454,
  2137242950, 852679640,  1014566730, 1999335993, 1871390758, 1736439305,
  231222289,  603972436,  783045542,  370384393,  184356284,  709706295,
  1453549767, 591603172,  768512391,  854125182
]

@cython.boundscheck(False)
@cython.wraparound(False)
def chunker_cython(const uint8_t[:] data):
    cdef uint32_t pattern, mask, c, i, length
    pattern = 0
    mask = 32767
    c = 0
    length = data.shape[0]
    with nogil:
        for i in range(length):
            pattern = (pattern >> 1) + GEAR_C[data[i]]
            if not pattern & mask:
                c += 1
    return c

## Benchmark Chunking Operations

In [24]:
def benchmark_chunker(func, data):
    func_name = func.__name__
    num_bytes = len(data)
    data_size = humanize.naturalsize(num_bytes)
    t = Timer(logger=None)
    t.start()
    result = func(data)
    t.stop()
    print(f"{data_size} with {func_name} : {humanize.naturalsize(num_bytes / t.last)}/s (total={t.last:.4f}s result={result})")

DATA = os.urandom(1024*512*100)
benchmark_chunker(chunker_simple, DATA)
benchmark_chunker(chunker_cython, DATA)

52.4 MB with chunker_simple : 6.4 MB/s (total=8.2542s result=1587)
52.4 MB with chunker_cython : 1.8 GB/s (total=0.0297s result=1587)


## Parallel Chunking Operations

In [25]:
def benchmark_multi_chunker(func):
    func_name = func.__name__
    num_bytes = os.path.getsize(DUMMY)    
    data_size = humanize.naturalsize(num_bytes)
    t = Timer(logger=None)
    t.start()
    result = func()
    t.stop()
    print(f"{data_size} with {func_name} : {humanize.naturalsize(num_bytes / t.last)}/s (total={t.last:.4f}s result={result[:3]}...{result[-3:]})")

def fixed_chunks(chunk_size=1024*512):
    with open(DUMMY, 'rb') as f:
        data = f.read(chunk_size)
        while data:
            yield data
            data = f.read(chunk_size)
            
def serial_chunker():
    result = []
    for chunk in fixed_chunks():
        result.append(chunker_cython(chunk))
    return result

def parallel_chunker():
    with ThreadPoolExecutor(max_workers=8) as ex:
        jobs = [ex.submit(chunker_cython, chunk) for chunk in fixed_chunks()]
    return [job.result() for job in jobs]

benchmark_multi_chunker(serial_chunker)
benchmark_multi_chunker(parallel_chunker)

1.0 GB with serial_chunker : 1.3 GB/s (total=0.7719s result=[14, 15, 11]...[21, 23, 7])
1.0 GB with parallel_chunker : 3.3 GB/s (total=0.3000s result=[14, 15, 11]...[21, 23, 7])


# Helper functions

### ceil_div

In [26]:
def ceil_div_py(x, y):
    return (x + y - 1) // y

In [27]:
%%cython
cimport cython
from libc.stdint cimport uint32_t
cpdef uint32_t ceil_div_cy(uint32_t x, uint32_t y):
    return (x + y - 1) // y

In [28]:
print(ceil_div_py(10, 5))
print(ceil_div_cy(10, 5))
%timeit ceil_div_py(10, 5)
%timeit ceil_div_cy(10, 5)

2
2
121 ns ± 7 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
88.2 ns ± 14.5 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


### logarithm2

In [29]:
import math
def logarithm2_py(value: int) -> int:
    return round(math.log(value, 2))

In [30]:
%%cython
cimport cython
from libc.stdint cimport uint32_t
from libc.math cimport log2, lround

cpdef uint32_t logarithm2_cy(uint32_t value):
    return lround(log2(value))

In [31]:
print(logarithm2_py(32769))
print(logarithm2_cy(32769))
%timeit logarithm2_py(32769)
%timeit logarithm2_cy(32769)

15
15
397 ns ± 80.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
86.2 ns ± 4.48 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


### mask

In [32]:
def mask_py(bits):
    assert bits >= 1
    assert bits <= 31
    return 2 ** bits - 1

In [33]:
%%cython
cimport cython
from libc.stdint cimport uint32_t

cpdef uint32_t mask_cy(uint32_t bits):
    assert bits >= 1
    assert bits <= 31
    return 2 ** bits - 1

In [34]:
print(mask_py(24))
print(mask_cy(24))
%timeit mask_py(24)
%timeit mask_cy(24)

16777215
16777215
304 ns ± 18.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
59.5 ns ± 6.3 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


### center_size

In [35]:
def center_size_py(average: int, minimum: int, source_size: int) -> int:
    offset = minimum + ceil_div_py(minimum, 2)
    if offset > average:
        offset = average
    size = average - offset
    if size > source_size:
        return source_size
    else:
        return size

In [36]:
%%cython
cimport cython
from libc.stdint cimport uint32_t

cdef uint32_t ceil_div_cy(uint32_t x, uint32_t y):
    return (x + y - 1) // y

cpdef uint32_t center_size_cy(uint32_t average, uint32_t minimum, uint32_t source_size):
    cdef uint32_t offset = minimum + ceil_div_cy(minimum, 2)
    if offset > average:
        offset = average
    cdef uint32_t size = average - offset
    if size > source_size:
        return source_size
    return size

In [37]:
print(center_size_py(200, 100, 50))
print(center_size_cy(200, 100, 50))
print(center_size_cy(200, 100, 2000))
%timeit center_size_py(200, 100, 50)
%timeit center_size_cy(200, 100, 50)

50
50
50
268 ns ± 20.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
88.6 ns ± 4.87 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


# FastCDC Single Offset

## Python CDC Offset Calculation

In [38]:
def cdc_offset_py(data, mi, av, ma, cs, mask_s, mask_l):
    """Returns offset to first chunkpoint in data."""
    pattern = 0
    i = mi
    size = len(data)
    barrier = min(cs, size)  
    while i < barrier:
        pattern = (pattern >> 1) + GEAR[data[i]]
        if not pattern & mask_s:
            return i + 1
        i += 1
    barrier = min(ma, size)
    while i < barrier:
        pattern = (pattern >> 1) + GEAR[data[i]]
        if not pattern & mask_l:
            return i + 1
        i += 1
    return i

## Cython CDC Offset Calculation

In [39]:
%%cython
cimport cython
from libc.stdint cimport uint32_t, uint8_t

cdef uint32_t[256] GEAR_C = [
  1553318008, 574654857,  759734804,  310648967,  1393527547, 1195718329,
  694400241,  1154184075, 1319583805, 1298164590, 122602963,  989043992,
  1918895050, 933636724,  1369634190, 1963341198, 1565176104, 1296753019,
  1105746212, 1191982839, 1195494369, 29065008,   1635524067, 722221599,
  1355059059, 564669751,  1620421856, 1100048288, 1018120624, 1087284781,
  1723604070, 1415454125, 737834957,  1854265892, 1605418437, 1697446953,
  973791659,  674750707,  1669838606, 320299026,  1130545851, 1725494449,
  939321396,  748475270,  554975894,  1651665064, 1695413559, 671470969,
  992078781,  1935142196, 1062778243, 1901125066, 1935811166, 1644847216,
  744420649,  2068980838, 1988851904, 1263854878, 1979320293, 111370182,
  817303588,  478553825,  694867320,  685227566,  345022554,  2095989693,
  1770739427, 165413158,  1322704750, 46251975,   710520147,  700507188,
  2104251000, 1350123687, 1593227923, 1756802846, 1179873910, 1629210470,
  358373501,  807118919,  751426983,  172199468,  174707988,  1951167187,
  1328704411, 2129871494, 1242495143, 1793093310, 1721521010, 306195915,
  1609230749, 1992815783, 1790818204, 234528824,  551692332,  1930351755,
  110996527,  378457918,  638641695,  743517326,  368806918,  1583529078,
  1767199029, 182158924,  1114175764, 882553770,  552467890,  1366456705,
  934589400,  1574008098, 1798094820, 1548210079, 821697741,  601807702,
  332526858,  1693310695, 136360183,  1189114632, 506273277,  397438002,
  620771032,  676183860,  1747529440, 909035644,  142389739,  1991534368,
  272707803,  1905681287, 1210958911, 596176677,  1380009185, 1153270606,
  1150188963, 1067903737, 1020928348, 978324723,  962376754,  1368724127,
  1133797255, 1367747748, 1458212849, 537933020,  1295159285, 2104731913,
  1647629177, 1691336604, 922114202,  170715530,  1608833393, 62657989,
  1140989235, 381784875,  928003604,  449509021,  1057208185, 1239816707,
  525522922,  476962140,  102897870,  132620570,  419788154,  2095057491,
  1240747817, 1271689397, 973007445,  1380110056, 1021668229, 12064370,
  1186917580, 1017163094, 597085928,  2018803520, 1795688603, 1722115921,
  2015264326, 506263638,  1002517905, 1229603330, 1376031959, 763839898,
  1970623926, 1109937345, 524780807,  1976131071, 905940439,  1313298413,
  772929676,  1578848328, 1108240025, 577439381,  1293318580, 1512203375,
  371003697,  308046041,  320070446,  1252546340, 568098497,  1341794814,
  1922466690, 480833267,  1060838440, 969079660,  1836468543, 2049091118,
  2023431210, 383830867,  2112679659, 231203270,  1551220541, 1377927987,
  275637462,  2110145570, 1700335604, 738389040,  1688841319, 1506456297,
  1243730675, 258043479,  599084776,  41093802,   792486733,  1897397356,
  28077829,   1520357900, 361516586,  1119263216, 209458355,  45979201,
  363681532,  477245280,  2107748241, 601938891,  244572459,  1689418013,
  1141711990, 1485744349, 1181066840, 1950794776, 410494836,  1445347454,
  2137242950, 852679640,  1014566730, 1999335993, 1871390758, 1736439305,
  231222289,  603972436,  783045542,  370384393,  184356284,  709706295,
  1453549767, 591603172,  768512391,  854125182
]

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef uint32_t cdc_offset_cy(const uint8_t[:] data, const uint32_t mi, const uint32_t av, const uint32_t ma, const uint32_t cs, const uint32_t mask_s, const uint32_t mask_l):
    cdef uint32_t pattern, i, size, barrier
    pattern = 0
    i = mi
    size = len(data)
    barrier = min(cs, size)  
    while i < barrier:
        pattern = (pattern >> 1) + GEAR_C[data[i]]
        if not pattern & mask_s:
            return i + 1
        i += 1
    barrier = min(ma, size)
    while i < barrier:
        pattern = (pattern >> 1) + GEAR_C[data[i]]
        if not pattern & mask_l:
            return i + 1
        i += 1
    return i

## Benchmark CDC Offset Calculation

In [40]:
av = 16384
mi = av // 4
ma = av * 8
cs = center_size_py(av, mi, ma)
bits = logarithm2_py(av)
mask_s = mask_py(bits + 1)
mask_l = mask_py(bits - 1)
data = os.urandom(1000000)
print(cdc_offset_py(data, mi, av, ma, cs, mask_s, mask_l))
print(cdc_offset_cy(data, mi, av, ma, cs, mask_s, mask_l))
%timeit cdc_offset_py(data, mi, av, ma, cs, mask_s, mask_l)
%timeit cdc_offset_cy(data, mi, av, ma, cs, mask_s, mask_l)

16346
16346
2.7 ms ± 227 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.49 µs ± 282 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [41]:
def fastcdc_py(stream, avg_size=16384, read_size=1024*32):
    chunk_points = []
    av = avg_size
    mi = av // 4
    ma = av * 8
    bits = logarithm2_py(av)
    mask_s = mask_py(bits + 1)
    mask_l = mask_py(bits - 1)
    cs = center_size_py(av, mi, ma)
    data = stream.read(read_size)
    offset = 0
    while data:
        if len(data) <= ma:
            data += stream.read(read_size)
        cp = cdc_offset_cy(data, mi, av, ma, cs, mask_s, mask_l)
        chunk_points.append((offset, cp))
        offset += cp
        data = data[cp:]
    return chunk_points

In [42]:
import os
import io
num_bytes = 1000000 * 1000 # 100 MB
data = os.urandom(num_bytes) # 100 MB
stream = io.BytesIO(data)
def benchmark_chunker(func):
    func_name = func.__name__
    file_size = humanize.naturalsize(num_bytes)
    t = Timer(logger=None)
    t.start()
    result = func(stream)
    t.stop()
    print(f"{humanize.naturalsize(num_bytes / t.last)}/s (result: {result[0]} ... {result[-1]})")
benchmark_chunker(fastcdc_py)

1.0 GB/s (result: (0, 27829) ... (999994471, 5529))


# Objects

## Python Objects

In [43]:
from dataclasses import dataclass
@dataclass
class Chunk:
    offset: int
    length: int
    data: bytes
    hash_: str

def gen_objects_py():
    objs = []
    for x in range(1000):
        objs.append(Chunk(10,100, b'', "lkajdsflkajsdf"))
    return objs

## Cython Objects

In [44]:
%%cython

cdef class Chunk:
    cdef readonly int offset, length
    cdef readonly char* data
    cdef readonly str hash_
    
    def __init__(self, offset, length, data, hash_):
        self.offset = offset
        self.length = length
        self.data = data
        self.hash_ = hash_

cpdef gen_objects_cy():
    objs = []
    for x in range(1000):
        objs.append(Chunk(10, 100, b'', "lkajdsflkajsdf"))
    return objs

## Benchmark Objects

In [45]:
%timeit gen_objects_py()
%timeit gen_objects_cy()

138 µs ± 6.24 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
42.7 µs ± 5.87 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# FastCDC Multiple Chunks

## Python with while loop

In [63]:
def cdc_while_py(data, mi, av, ma, cs, mask_s, mask_l):
    """
    Returns offsets to all chunkpoints in data.
    """
    chunks = []
    offset = 0
    section = data[offset:offset + ma]
    
    while True:
        found = False
        size = len(section)
        if size < ma:
            section = data[offset:offset + ma]
            size = len(section)
        if size == 0:
            break

        i = mi
        pattern = 0
        if size <= mi:
            chunks.append(size)
            break
        
        barrier = min(cs, size)  
        while i < barrier:
            pattern = (pattern >> 1) + GEAR[section[i]]
            if not pattern & mask_s:
                offset += i+1
                chunks.append(offset)
                section = data[offset:offset + ma]
                found = True
                break
            i += 1
        if found:
            continue
        barrier = min(ma, size)
        while i < barrier:
            pattern = (pattern >> 1) + GEAR[section[i]]
            if not pattern & mask_l:
                offset += i+1
                chunks.append(offset)
                section = data[offset:offset + ma]
                found = True
                break
            i += 1
        if found:
            continue
        
        offset += i
        section = data[offset:offset + ma]
    return chunks

## Benchmark Multicple Chunks

In [64]:
av = 8192
mi = av // 4
ma = av * 8
cs = center_size_py(av, mi, ma)
bits = logarithm2_py(av)
mask_s = mask_py(bits + 1)
mask_l = mask_py(bits - 1)
data = open('../tests/SekienAkashita.jpg', 'rb').read()
print(cdc_while_py(data, mi, av, ma, cs, mask_s, mask_l))
%timeit cdc_while_py(data, mi, av, ma, cs, mask_s, mask_l)

[22366, 30116, 32857, 43588, 50717, 65647, 86053, 92136, 104063]
19.2 ms ± 1.26 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
